# Quick start with Pandas in 45 mins:
## data structures, IO, aggregation, melt/pivot, group operations, merge/join

this basically covers (too fast) the concepts you need to master with Pandas

Author: Alexandre Gramfort
(based on Wes McKinney's book "Python for Data Analysis" O'REILLY)

In [1]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision=4)

In [2]:
pd.options.display.notebook_repr_html = False

In [3]:
%matplotlib inline

## Pandas objects : DataFrame, Series, Index

In [4]:
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
df

      data1     data2 key1 key2
0 -0.204708  1.393406    a  one
1  0.478943  0.092908    a  two
2 -0.519439  0.281746    b  one
3 -0.555730  0.769023    b  two
4  1.965781  1.246435    a  one

In [5]:
type(df)  # df is a DataFrame

pandas.core.frame.DataFrame

In [6]:
df.dtypes  # one dtype per column (strings are object)

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [7]:
data1 = df['data1']  # extract column 'data1'
data1

0   -0.204708
1    0.478943
2   -0.519439
3   -0.555730
4    1.965781
Name: data1, dtype: float64

In [8]:
type(data1)  # each column of a DataFrame is a Series

pandas.core.series.Series

A dataframe is basically a numpy array with names of columns and rows

The names of columns or rows are **both** an Index. Many different types of index exist (Index, RangeIndex, DatetimeIndex etc.)

In [9]:
df.index

RangeIndex(start=0, stop=5, step=1)

In [10]:
df.columns

Index(['data1', 'data2', 'key1', 'key2'], dtype='object')

To get the values as a Numpy ndarray just do

In [11]:
values = df.values
print(values)

[[-0.20470765948471295 1.3934058329729904 'a' 'one']
 [0.47894333805754824 0.09290787674371767 'a' 'two']
 [-0.5194387150567381 0.2817461528302025 'b' 'one']
 [-0.55573030434749 0.7690225676118387 'b' 'two']
 [1.9657805725027142 1.2464347363862822 'a' 'one']]


In [12]:
print(type(values))

<class 'numpy.ndarray'>


In [13]:
values.dtype, values.ndim, values.shape

(dtype('O'), 2, (5, 4))

In [14]:
values[:, :2].astype(float)  # get the first two columns and make them float

array([[-0.2047,  1.3934],
       [ 0.4789,  0.0929],
       [-0.5194,  0.2817],
       [-0.5557,  0.769 ],
       [ 1.9658,  1.2464]])

Some basic methods of a DataFrame

In [15]:
df.count()

data1    5
data2    5
key1     5
key2     5
dtype: int64

In [16]:
df.sort_values(by='data1')

      data1     data2 key1 key2
3 -0.555730  0.769023    b  two
2 -0.519439  0.281746    b  one
0 -0.204708  1.393406    a  one
1  0.478943  0.092908    a  two
4  1.965781  1.246435    a  one

In [17]:
df.min()  # you can do max, mean etc.

data1     -0.55573
data2    0.0929079
key1             a
key2           one
dtype: object

In [18]:
df['key1'].unique()  # get unique entries of a column

array(['a', 'b'], dtype=object)

In [19]:
for col in df:  # loop over columns
    print(df[col].unique())

[-0.2047  0.4789 -0.5194 -0.5557  1.9658]
[ 1.3934  0.0929  0.2817  0.769   1.2464]
['a' 'b']
['one' 'two']


Making your DataFrame all numerical (for learning)

In [20]:
df_dummies = pd.get_dummies(df, columns=['key1', 'key2'])  # dummify
df_dummies

      data1     data2  key1_a  key1_b  key2_one  key2_two
0 -0.204708  1.393406     1.0     0.0       1.0       0.0
1  0.478943  0.092908     1.0     0.0       0.0       1.0
2 -0.519439  0.281746     0.0     1.0       1.0       0.0
3 -0.555730  0.769023     0.0     1.0       0.0       1.0
4  1.965781  1.246435     1.0     0.0       1.0       0.0

In [21]:
df_dummies.values.dtype  # now we have only floats !

dtype('float64')

Or factorize (make column a factor)

In [22]:
values, index = pd.factorize(df['key1'])
values, index

(array([0, 0, 1, 1, 0]), Index(['a', 'b'], dtype='object'))

In [23]:
df_factorized = df.copy()
df_factorized['key1'] = pd.factorize(df['key1'])[0]
df_factorized['key2'] = pd.factorize(df['key2'])[0]
df_factorized

      data1     data2  key1  key2
0 -0.204708  1.393406     0     0
1  0.478943  0.092908     0     1
2 -0.519439  0.281746     1     0
3 -0.555730  0.769023     1     1
4  1.965781  1.246435     0     0

In [24]:
df_factorized.values.dtype

dtype('float64')

## Basic indexing / Selection

In [25]:
row = df.ix[1]
row

data1     0.478943
data2    0.0929079
key1             a
key2           two
Name: 1, dtype: object

In [26]:
type(row)

pandas.core.series.Series

In [27]:
row.dtype

dtype('O')

In [28]:
df.ix[2:3, ['key1', 'data1']]

  key1     data1
2    b -0.519439
3    b -0.555730

In [29]:
df.ix[:2]  # WARNING 2 is included !

      data1     data2 key1 key2
0 -0.204708  1.393406    a  one
1  0.478943  0.092908    a  two
2 -0.519439  0.281746    b  one

In [30]:
df.values[:2]  # vs 2 not included in numpy !

array([[-0.20470765948471295, 1.3934058329729904, 'a', 'one'],
       [0.47894333805754824, 0.09290787674371767, 'a', 'two']], dtype=object)

In [31]:
df.ix[:, :2]  # get the first two columns

      data1     data2
0 -0.204708  1.393406
1  0.478943  0.092908
2 -0.519439  0.281746
3 -0.555730  0.769023
4  1.965781  1.246435

Efficiently select with a condition

In [68]:
df[df['data1'] > 0.]

      data1     data2 key1 key2
0  0.886429 -0.539741    a  one
3  1.669025 -1.021228    b  two

In [69]:
df[df['key1'] == 'a']

      data1     data2 key1 key2
0  0.886429 -0.539741    a  one
1 -2.001637  0.476985    a  two
4 -0.438570 -0.577087    a  one

## Algebra with DataFrames

In [32]:
df1 = df[['data1', 'data2']]  # extract a subset of columns
df1

      data1     data2
0 -0.204708  1.393406
1  0.478943  0.092908
2 -0.519439  0.281746
3 -0.555730  0.769023
4  1.965781  1.246435

In [33]:
df1 + 100  # addition

        data1       data2
0   99.795292  101.393406
1  100.478943  100.092908
2   99.480561  100.281746
3   99.444270  100.769023
4  101.965781  101.246435

In [34]:
df1 * df1  # element wise multiplication

      data1     data2
0  0.041905  1.941580
1  0.229387  0.008632
2  0.269817  0.079381
3  0.308836  0.591396
4  3.864293  1.553600

In [35]:
df1 ** 2  # power

      data1     data2
0  0.041905  1.941580
1  0.229387  0.008632
2  0.269817  0.079381
3  0.308836  0.591396
4  3.864293  1.553600

In [36]:
# make a new df with an incremented index
df2 = DataFrame({'data3' : np.random.randn(5)}, index=df1.index + 1)
df2

      data3
1  1.007189
2 -1.296221
3  0.274992
4  0.228913
5  1.352917

In [37]:
df1 + df2  # you get NaNs because columns don't match

   data1  data2  data3
0    NaN    NaN    NaN
1    NaN    NaN    NaN
2    NaN    NaN    NaN
3    NaN    NaN    NaN
4    NaN    NaN    NaN
5    NaN    NaN    NaN

In [38]:
df3 = pd.concat([df1, df2], axis=1)  # WARNING : index matters !
df3

      data1     data2     data3
0 -0.204708  1.393406       NaN
1  0.478943  0.092908  1.007189
2 -0.519439  0.281746 -1.296221
3 -0.555730  0.769023  0.274992
4  1.965781  1.246435  0.228913
5       NaN       NaN  1.352917

In [39]:
df3.shape

(6, 3)

In [40]:
df3.count()

data1    5
data2    5
data3    5
dtype: int64

In [41]:
df3.describe()

/Users/alex/anaconda/envs/py35/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


          data1     data2     data3
count  5.000000  5.000000  5.000000
mean   0.232969  0.756703  0.313558
std    1.053905  0.572623  1.019979
min   -0.555730  0.092908 -1.296221
25%         NaN       NaN       NaN
50%         NaN       NaN       NaN
75%         NaN       NaN       NaN
max    1.965781  1.393406  1.352917

In [42]:
df3.dropna()  # drop NaNs

      data1     data2     data3
1  0.478943  0.092908  1.007189
2 -0.519439  0.281746 -1.296221
3 -0.555730  0.769023  0.274992
4  1.965781  1.246435  0.228913

In [43]:
df3.dropna().describe()  # get descriptive statistifs from numeric columns

          data1     data2     data3
count  4.000000  4.000000  4.000000
mean   0.342389  0.597528  0.053718
std    1.183697  0.517961  0.968005
min   -0.555730  0.092908 -1.296221
25%   -0.528512  0.234537 -0.152371
50%   -0.020248  0.525384  0.251952
75%    0.850653  0.888376  0.458041
max    1.965781  1.246435  1.007189

## IO : Input Output

In [44]:
df.to_csv('data.csv')  # save to CSV (with index)

In [45]:
file_content = open('data.csv').read()
print(file_content)

,data1,data2,key1,key2
0,-0.20470765948471295,1.3934058329729904,a,one
1,0.47894333805754824,0.09290787674371767,a,two
2,-0.5194387150567381,0.2817461528302025,b,one
3,-0.55573030434749,0.7690225676118387,b,two
4,1.9657805725027142,1.2464347363862822,a,one



In [46]:
df.to_csv('data.csv', index=False)  # save to CSV (without index)
file_content = open('data.csv').read()
print(file_content)

data1,data2,key1,key2
-0.20470765948471295,1.3934058329729904,a,one
0.47894333805754824,0.09290787674371767,a,two
-0.5194387150567381,0.2817461528302025,b,one
-0.55573030434749,0.7690225676118387,b,two
1.9657805725027142,1.2464347363862822,a,one



Read again the data

In [47]:
df = pd.read_csv('data.csv')

In [48]:
df

      data1     data2 key1 key2
0 -0.204708  1.393406    a  one
1  0.478943  0.092908    a  two
2 -0.519439  0.281746    b  one
3 -0.555730  0.769023    b  two
4  1.965781  1.246435    a  one

## GroupBy mechanics

In [49]:
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
df

      data1     data2 key1 key2
0  0.886429 -0.539741    a  one
1 -2.001637  0.476985    a  two
2 -0.371843  3.248944    b  one
3  1.669025 -1.021228    b  two
4 -0.438570 -0.577087    a  one

In [50]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [51]:
grouped.size()

key1
a    3
b    2
dtype: int64

In [52]:
grouped.mean()

key1
a   -0.517926
b    0.648591
Name: data1, dtype: float64

In [53]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.223930
      two    -2.001637
b     one    -0.371843
      two     1.669025
Name: data1, dtype: float64

In [54]:
means.index  # See the hierarchical

MultiIndex(levels=[['a', 'b'], ['one', 'two']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['key1', 'key2'])

In [55]:
means.unstack()

key2       one       two
key1                    
a     0.223930 -2.001637
b    -0.371843  1.669025

### Selecting a column or subset of columns

In [56]:
df.groupby('key1')['data1']

In [57]:
df.groupby('key1')[['data2']]

In [58]:
df.groupby(['key1', 'key2'])[['data2']].mean()

              data2
key1 key2          
a    one  -0.558414
     two   0.476985
b    one   3.248944
     two  -1.021228

### Grouping by index levels

In [59]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]], names=['cty', 'tenor'])
hier_df = DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.124121  0.302614  0.523772  0.000940  1.343810
1     -0.713544 -0.831154 -2.370232 -1.860761 -0.860757
2      0.560145 -1.265934  0.119827 -1.063512  0.332883
3     -2.359419 -0.199543 -1.541996 -0.970736 -1.307030

In [60]:
hier_df.groupby(level='cty', axis=1).count()

cty  JP  US
0     2   3
1     2   3
2     2   3
3     2   3

## melt and pivot_table

melt is useful to "flatten" a DataFrame and pivot_table to "pivot" it. Let's see an example.

In [61]:
hier_df_flat = pd.melt(hier_df)  # flatten the DataFrame
hier_df_flat  # note that you loose the index

   cty  tenor     value
0   US      1  0.124121
1   US      1 -0.713544
2   US      1  0.560145
3   US      1 -2.359419
4   US      3  0.302614
5   US      3 -0.831154
6   US      3 -1.265934
7   US      3 -0.199543
8   US      5  0.523772
9   US      5 -2.370232
10  US      5  0.119827
11  US      5 -1.541996
12  JP      1  0.000940
13  JP      1 -1.860761
14  JP      1 -1.063512
15  JP      1 -0.970736
16  JP      3  1.343810
17  JP      3 -0.860757
18  JP      3  0.332883
19  JP      3 -1.307030

In [62]:
hier_df_flat = pd.melt(hier_df.reset_index(), id_vars=['index'])  # now keep the index
hier_df_flat

    index cty tenor     value
0       0  US     1  0.124121
1       1  US     1 -0.713544
2       2  US     1  0.560145
3       3  US     1 -2.359419
4       0  US     3  0.302614
5       1  US     3 -0.831154
6       2  US     3 -1.265934
7       3  US     3 -0.199543
8       0  US     5  0.523772
9       1  US     5 -2.370232
10      2  US     5  0.119827
11      3  US     5 -1.541996
12      0  JP     1  0.000940
13      1  JP     1 -1.860761
14      2  JP     1 -1.063512
15      3  JP     1 -0.970736
16      0  JP     3  1.343810
17      1  JP     3 -0.860757
18      2  JP     3  0.332883
19      3  JP     3 -1.307030

Use pivot_table to undo the melt.

In [63]:
hier_df_flat.pivot_table(index='index', columns=['cty', 'tenor'])

          value                                        
cty          JP                  US                    
tenor         1         3         1         3         5
index                                                  
0      0.000940  1.343810  0.124121  0.302614  0.523772
1     -1.860761 -0.860757 -0.713544 -0.831154 -2.370232
2     -1.063512  0.332883  0.560145 -1.265934  0.119827
3     -0.970736 -1.307030 -2.359419 -0.199543 -1.541996

## Group-wise operations and transformations

In [64]:
df

      data1     data2 key1 key2
0  0.886429 -0.539741    a  one
1 -2.001637  0.476985    a  two
2 -0.371843  3.248944    b  one
3  1.669025 -1.021228    b  two
4 -0.438570 -0.577087    a  one

In [65]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')
k1_means

      mean_data1  mean_data2
key1                        
a      -0.517926   -0.213281
b       0.648591    1.113858

In [66]:
pd.merge(df, k1_means, left_on='key1', right_index=True)

      data1     data2 key1 key2  mean_data1  mean_data2
0  0.886429 -0.539741    a  one   -0.517926   -0.213281
1 -2.001637  0.476985    a  two   -0.517926   -0.213281
4 -0.438570 -0.577087    a  one   -0.517926   -0.213281
2 -0.371843  3.248944    b  one    0.648591    1.113858
3  1.669025 -1.021228    b  two    0.648591    1.113858

# To learn more:

- https://github.com/jakevdp/PythonDataScienceHandbook (Chapter 3)
- https://github.com/wesm/pydata-book (esp. Chapter 9)
- http://nbviewer.jupyter.org/github/fonnesbeck/Bios8366/blob/master/notebooks/Section2_1-Introduction-to-Pandas.ipynb
- https://github.com/jvns/pandas-cookbook
- http://www.dataschool.io/best-python-pandas-resources/
